In [1]:
import pandas as pd
from langchain_community.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain_core.documents import Document
import os

## 筛选跟每个service program 相关的posts

In [3]:
# Directory path
directory_path = 'senti_results_SZ'
all_docs = []

# Iterate through CSV files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Use pandas to read the CSV and preserve row information
        df = pd.read_csv(file_path)
        
        # Convert each row to a Document with appropriate metadata
        for index, row in df.iterrows():
            # Convert row to string representation
            content = row.to_string()
            # Create metadata with file source and row index
            metadata = {
                'source': file_path,
                'row': index,
                # You can add other metadata from your CSV if needed
            }
            doc = Document(page_content=content, metadata=metadata)
            all_docs.append(doc)

print(f"Loaded {len(all_docs)} documents (rows) from CSVs")

Loaded 58738 documents (rows) from CSVs


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts_all = text_splitter.split_documents(all_docs)

In [5]:
hg_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [ ]:
# persist_directory = 'docs/chroma_rag/'
# posts_langchain_chroma = Chroma.from_documents(
#     documents=texts_all,
#     collection_name="SZ_posts_senti",
#     embedding=hg_embeddings,
#     persist_directory=persist_directory
# )

In [ ]:
persist_directory = 'docs/chroma_rag/'
posts_langchain_chroma = Chroma(
    collection_name="SZ_posts_senti",
    persist_directory=persist_directory,
    embedding_function=hg_embeddings
)

#### Description Data for Service Program

In [7]:
pd.read_csv("service_program_data/SPD_SZ_zh.csv").head()

,Related service quality,Service improvement programs,Objective,City,Period,Investments,Reference
0,舒适度-温度,同一车厢不同温度模式,NaN,深圳,2020-01-01至2021-12-31,不适用,https://www.szmc.net/SMARTC/upload/file/202106...
1,信息服务,智能动态地图显示系统,NaN,深圳,2020-01-01至2021-12-31,不适用,https://www.szmc.net/SMARTC/upload/file/202106...
2,信息服务,车厢拥挤度智能显示系统,智能拥挤度系统将在5条线路上推出：6、7、9、10和11号线,深圳,2020-01-01至2021-12-31,不适用,https://www.szmc.net/SMARTC/upload/file/202106...
3,配套设施-票务服务,实现BOM第三方支付,NaN,深圳,2020-01-01至2021-12-31,不适用,https://www.szmc.net/SMARTC/upload/file/202106...
4,配套设施-票务服务,"成功推出""深圳通""乘车码",NaN,深圳,2020-01-01至2021-12-31,不适用,https://www.szmc.net/SMARTC/upload/file/202106...


In [18]:
import pandas as pd
from IPython.display import display, HTML

# Load the CSV file
service_program_df = pd.read_csv('service_program_data/SPD_SZ_zh.csv')

# Create a list to store results
results = []

# Iterate through each row of the CSV
for index, row in service_program_df.iterrows():
    # Concatenate the first three columns as service_dimension
    service_dimension = ' '.join([str(row[col]) for col in service_program_df.columns[:3]])
    
    # Search for similar documents in the vector database
    docs = posts_langchain_chroma.similarity_search(service_dimension, k=100)  # You can adjust k as needed
    
    # Store the results
    result = {
        'service_dimension': service_dimension,
        'similar_documents': docs
    }
    results.append(result)
    
    # Print progress (optional)
    if index % 10 == 0:
        print(f"Processed {index} rows")

# Display some results (for example, the first one)
if results:
    for i in range(len(results)):
      print(f"Service dimension: {results[i]['service_dimension']}")
      print("\nSimilar documents:")
      for j, doc in enumerate(results[i]['similar_documents']):
          if j < 10:
            print(f"\nDocument {j+1}:")
            print(f"Content: {doc.page_content[200:]}...")  # Display first 200 chars
            print(f"Metadata: {doc.metadata}")
            print(f"Metadata: {doc.metadata.get('source', 'N/A')}, Row: {doc.metadata.get('row', 'N/A')}")
      print("\n")

Processed 0 rows
Processed 10 rows
Processed 20 rows
Service dimension: 舒适度-温度 同一车厢不同温度模式 nan

Similar documents:

Document 1:
Content:                                                   庄哥实验室
微博正文                     深圳的地铁在两头都设置了女士优先车厢，这些车厢空调温度相对会好点，不会太冷！
头条文章url                                                     NaN
发布位置                                                         深圳
艾特用户                                                        NaN
话题                                                          NaN
转发数                                                           3
评论数                                                           7
点赞数                                                           8
发布时间                                            2022/7/31 13:52
发布工具                                                        NaN
微博图片url       ['https://wx1.sinaimg.cn/large/006YIcbEgy1h4q2......
Metadata: {'row': 4, 'source': 'senti_results_SZ/senti_cleaned_深圳 地铁 202207 1.0.csv'}
Metadata: senti

In [10]:
df = pd.read_csv('senti_results_SZ/senti_cleaned_深圳 地铁 201901 1.0.csv')
df.head(5)

,id,bid,user_id,用户昵称,微博正文,头条文章url,发布位置,艾特用户,话题,转发数,...,点赞数,发布时间,发布工具,微博图片url,微博视频url,retweet_id,ip,Negative,Neutral,Positive
0,4.320000e+15,H9Z8kpBtd,5847892183,xieqingheng,赶在末班车回去，挺幸运的#深圳地铁#,NaN,深圳·民治,NaN,深圳地铁,0,...,0,2019/1/1 23:46,iPhone客户端,NaN,NaN,NaN,NaN,0.142750,0.229226,0.917541
1,4.320000e+15,H9Z6e7pT9,5390310654,蚊子嚎叫,2019年第一天晚，深圳地铁温馨提供凌晨地铁，让我终于今天加班不用再打的回去了,NaN,NaN,NaN,NaN,0,...,1,2019/1/1 23:41,小米8青春版 潮流旗舰,['https://wx3.sinaimg.cn/large/005SNc2qly1fyri...,NaN,NaN,NaN,0.150500,0.353703,0.876308
2,4.330000e+15,HcRYBm5j4,1900132663,Hey朱zai,在周末回港的最后一刻丢了深圳地铁卡和香港八达通(已经陪伴我快六年的学生卡），以及非常喜欢的卡...,NaN,NaN,NaN,NaN,0,...,0,2019/1/20 23:04,💌iPhone 7 Plus,['https://wx1.sinaimg.cn/large/7141b937gy1fzdg...,NaN,NaN,NaN,0.946389,0.247320,0.194873
3,4.330000e+15,HcRWx7arv,6365144629,LavenderNights,啊啊啊啊啊啊啊要是深圳地铁applepay可以跟上海一样用交通卡就好了…不想弄银联闪付啊并不...,NaN,NaN,NaN,NaN,0,...,0,2019/1/20 22:59,iPhone客户端,NaN,NaN,NaN,NaN,0.926535,0.276487,0.228433
4,4.330000e+15,HcIvchoo0,1957317823,Kkkkks2,#顺德智障工程#深圳在维修地铁的时候，会有洒水车经常走过。但是顺德容桂连条铁都没有，却学人家...,NaN,佛山·容桂街区,NaN,顺德智障工程,1,...,2,2019/1/19 22:57,坚果手机 Pro 2,NaN,http://f.us.sinaimg.cn/003SxzFclx07qTBwOnwc010...,NaN,NaN,0.931672,0.274792,0.209922


In [26]:
import pandas as pd
from IPython.display import display, HTML
import os
import glob

# Create directory for output if it doesn't exist
output_dir = 'service_program_matches_SZ'
os.makedirs(output_dir, exist_ok=True)

# Load the service program CSV file
service_program_df = pd.read_csv('service_program_data/SPD_SZ_zh.csv')

# Create a cache for the source dataframes to avoid reloading them
source_df_cache = {}

# Iterate through each row of the CSV
for index, row in service_program_df.iterrows():
    # Concatenate the first three columns as service_dimension
    service_dimension = ' '.join([str(row[col]) for col in service_program_df.columns[:3]])
    
    # Search for similar documents in the vector database
    docs = posts_langchain_chroma.similarity_search(service_dimension, k=100)
    
    # Create a dictionary to group matched rows by source file
    matches_by_source = {}
    
    # Process each matched document
    for doc in docs:
        source = doc.metadata.get('source', None)
        row_index = doc.metadata.get('row', None)
        
        if source and row_index is not None:
            # Convert row to integer if it's a numeric string
            try:
                row_index = int(row_index)
            except (ValueError, TypeError):
                continue
                
            if source not in matches_by_source:
                matches_by_source[source] = []
            
            matches_by_source[source].append(row_index)
    
    # Create a list to store all matching rows from original sources
    all_matched_rows = []
    
    # For each source file, get the matching rows
    for source, row_indices in matches_by_source.items():
        # Load the source file if not in cache
        if source not in source_df_cache:
            try:
                source_df_cache[source] = pd.read_csv(source)
            except Exception as e:
                print(f"Error loading {source}: {e}")
                continue
        
        source_df = source_df_cache[source]
        
        # Get rows from original source file
        for row_idx in row_indices:
            try:
                if 0 <= row_idx < len(source_df):
                    row_data = source_df.iloc[row_idx].to_dict()
                    row_data['original_source'] = source
                    row_data['original_row'] = row_idx
                    all_matched_rows.append(row_data)
            except Exception as e:
                print(f"Error accessing row {row_idx} in {source}: {e}")
    
    # Convert to DataFrame
    if all_matched_rows:
        matches_df = pd.DataFrame(all_matched_rows)
        
        # Create a filename for this service program
        safe_filename = f"service_program_{index}_matches.csv"
        
        # Save to CSV
        matches_df.to_csv(os.path.join(output_dir, safe_filename), index=False)
    
    # Print progress
    if index % 10 == 0:
        print(f"Processed {index} rows")

print(f"Completed! All match files saved to {output_dir} directory.")

# Display summary of what was created
print(f"Created CSV files with matching posts:")
for i, filename in enumerate(sorted(os.listdir(output_dir))):
    if i < 10:  # Show first 10 examples
        try:
            match_df = pd.read_csv(os.path.join(output_dir, filename))
            print(f"- {filename}: {len(match_df)} matched original posts")
        except:
            print(f"- {filename}: Could not read file")

Processed 0 rows
Processed 10 rows
Processed 20 rows
Completed! All match files saved to service_program_matches_SZ directory.
Created CSV files with matching posts:
- service_program_0_matches.csv: 100 matched original posts
- service_program_10_matches.csv: 100 matched original posts
- service_program_11_matches.csv: 100 matched original posts
- service_program_12_matches.csv: 100 matched original posts
- service_program_13_matches.csv: 100 matched original posts
- service_program_14_matches.csv: 100 matched original posts
- service_program_15_matches.csv: 100 matched original posts
- service_program_16_matches.csv: 100 matched original posts
- service_program_17_matches.csv: 100 matched original posts
- service_program_18_matches.csv: 100 matched original posts
